## Trains a FeedForward Net on the MNIST dataset;
* Uses Cross-Entropy loss;
* Uses SGD as optimiser;
* I have set a small number of epochs according to the hardware of my local machine;
* There is some code commented out showing what to do if you have a GPU;

In [1]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from torch import nn

In [2]:
torch.manual_seed(42)

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"I will be using {device}\n")

I will be using cpu



/home/mvas/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 9010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [4]:
# load the data
train_data = datasets.MNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor() # PIL format to normalised tensor
)

test_data = datasets.MNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor()
)

In [5]:
# wrap a DataLoader around the dataset objects to be able to iterate
train_dataloader = DataLoader(train_data, batch_size=50, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=50)

In [6]:
img, label = train_data[0]
print(f"img is of size {img.size()}\n")

img is of size torch.Size([1, 28, 28])



## 1) Choose model class:

In [7]:
class FFNet(nn.Module):
    def __init__(self):
        super(FFNet,self).__init__()
        # flatten object to flatten 28x28 image to a 
        # 1D contiguous array;
        self.flatten = nn.Flatten()
        self.my_net = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10) # digits are from 0 to 9 so 10 elements;
        )
        
    # forward pass
    def forward(self, x):
        x = self.flatten(x)
        return self.my_net(x)

In [8]:
# instantiate model
model = FFNet()#.to(device) # move to GPU if you have one
print(model) # has __str__ method

FFNet(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (my_net): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## 2) Choose Loss:

In [9]:
loss_fn = nn.CrossEntropyLoss()

## 3) Choose opimizer:

In [10]:
# Go with Stochastic Gradient Descent
sgd_optim = torch.optim.SGD(model.parameters(), lr = 1e-3)

In [11]:
# define train loop
def train_loop(train_dataloader, model, loss_fn, optim):
    size = len(train_dataloader.dataset)
    
    for batch, (X,y) in enumerate(train_dataloader, start=1):
        
        # forward pass
        preds = model(X)
        loss = loss_fn(preds, y)
        
        # backward pass
        # gradients are accumulated so to prevent duble counting
        # clear them after each pass;
        optim.zero_grad()
        loss.backward()
        optim.step()
        
        # print progress
        if batch%100 == 0:
            loss, progress = loss.item(), batch*len(X)
            print(f"loss: {loss:.5f}\tprogress: {progress}/{size}")

In [12]:
# define test loop to show accuracy and loss after 1 epoch
def test_loop(test_dataloader, model, loss_fn):
    
    test_loss, n_correct = 0,0
    size = len(test_dataloader.dataset)
    
    # here I don't want to track gradients, since this
    # is just an evaluation step, to see model performance;
    with torch.no_grad():
        for (X,y) in test_dataloader:
            preds = model(X)
            loss = loss_fn(preds, y)
            
            test_loss += loss * len(X)
            n_correct += (preds.argmax(1) == y).sum().item()
    print(f"Test metrics\nAverage loss: {test_loss/size:.5f}   Accuracy: {n_correct/size:.5f}\n")

In [13]:
# If more compute, go higher with the epochs;
epoch = 10

for t in range(epoch):
    print(f"Epoch: {t+1}\n----------------------------------")
    train_loop(train_dataloader, model, loss_fn, sgd_optim)
    test_loop(test_dataloader, model, loss_fn)

Epoch: 1
----------------------------------
loss: 2.30110	progress: 5000/60000
loss: 2.29647	progress: 10000/60000
loss: 2.28587	progress: 15000/60000
loss: 2.28576	progress: 20000/60000
loss: 2.28133	progress: 25000/60000
loss: 2.26634	progress: 30000/60000
loss: 2.26030	progress: 35000/60000
loss: 2.25930	progress: 40000/60000
loss: 2.26655	progress: 45000/60000
loss: 2.23857	progress: 50000/60000
loss: 2.24953	progress: 55000/60000
loss: 2.24172	progress: 60000/60000
Test metrics
Average loss: 2.24181   Accuracy: 0.51380

Epoch: 2
----------------------------------
loss: 2.24509	progress: 5000/60000
loss: 2.22520	progress: 10000/60000
loss: 2.21068	progress: 15000/60000
loss: 2.22011	progress: 20000/60000
loss: 2.20978	progress: 25000/60000
loss: 2.21519	progress: 30000/60000
loss: 2.19611	progress: 35000/60000
loss: 2.17729	progress: 40000/60000
loss: 2.18405	progress: 45000/60000
loss: 2.15704	progress: 50000/60000
loss: 2.15058	progress: 55000/60000
loss: 2.12810	progress: 60000/